<a href="https://colab.research.google.com/github/kmk4444/-Calculating-potential-customer-revenue-using-rule-based-classification/blob/main/Haber_cekme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install requests
!pip install beautifulsoup4
!pip install pandas
!pip install datetime
!pip install win32com

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from openai import OpenAI  # OpenAI API için gerekli
import base64
import requests

# OpenAI API anahtarını ayarla
client = OpenAI(
    # This is the default and can be omitted
    api_key='sk-proj-',
)

# SerpAPI anahtarını ayarla
SERP_API_KEY = ""

# This function encodes the image to base64
def get_image_base64(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            return base64.b64encode(response.content).decode('utf-8')
        else:
            return None
    except Exception as e:
        print(f"Error fetching image from {image_url}: {e}")
        return None

# Google'da haber aramak için SerpAPI kullan
def search_aselsan_news(start_date, end_date):
    params = {
        "q": "Aselsan haber OR Aselsan news",
        "tbm": "nws",
        "tbs": f"cdr:1,cd_min:{start_date},cd_max:{end_date}",
        "api_key": SERP_API_KEY
    }
    url = "https://serpapi.com/search"
    response = requests.get(url, params=params)
    results = response.json()

    print("SerpAPI Yanıtı:", response.json())

    news_links = []
    if 'news_results' in results:
        for news in results['news_results']:
            news_links.append({
                'title': news['title'],
                'link': news['link'],
                'date': news['date'],
                'source': news['source'],  # Haber kaynağı eklendi
                'thumbnail': news['thumbnail']
            })
    return news_links

# Web sayfasından haber içeriğini çek
def scrape_news_content(news_url):
    try:
        response = requests.get(news_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')
            content = ' '.join([p.get_text() for p in paragraphs])
            return content
        else:
            return None
    except Exception as e:
        print(f"Error fetching {news_url}: {e}")
        return None


# OpenAI API ile haber içeriğini özetle (gpt-4 modelini kullanarak)
def summarize_content(content):
    try:
        # OpenAI API'yi kullanarak özetleme yapıyoruz
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{
                "role": "system",
                "content": "Sen yardımsever çeviri ve özet uzmanısın."
            }, {
                "role": "user",
                "content": f"Aşağıdaki yazıyı Türkçe ve maksimum 250 token olacak şekilde özetler misin? Bu özeti yöneticilerime ileteceğim.\n\n{content}"
            }],
            max_tokens=750,
            temperature=0.5
        )
        # Yanıtı kontrol edelim
        if response and response.choices:
            summary = response.choices[0].message.content.strip()  # Hatanın düzeltildiği yer
            return summary
        else:
            return "Özet oluşturulamadı."
    except Exception as e:
        print(f"Error during summarization: {e}")
        return "Özetleme sırasında bir hata oluştu."

# Raporu CSV formatında kaydet (sütunları ";" ile ayır)
def save_news_to_csv(news_data):
    df = pd.DataFrame(news_data)
    file_name = f"aselsan_news_report_{datetime.now().strftime('%Y-%m-%d')}.csv"
    df.to_csv(file_name, index=False, encoding='utf-8', sep=';')
    print(f"Rapor oluşturuldu: {file_name}")

# E-posta içeriği oluşturma (.eml formatında kaydetme)
def create_eml_file(news_data, start_date, end_date):
    msg = MIMEMultipart()
    msg['From'] = 'your_email@example.com'
    msg['To'] = 'recipient@example.com'
    msg['Subject'] = f"ASELSAN Bülteni ({start_date} - {end_date})"

    body = f"<h1>ASELSAN Bülteni</h1>"
    body += f"<p><strong>Tarih Aralığı:</strong> {start_date} - {end_date}</p>"

    for news in news_data:
        body += f"<h2>{news['Title']}</h2>"
        body += f"<p><strong>Kaynak:</strong> {news['Source']}</p>"

        # Embed the image using base64
        img_base64 = get_image_base64(news['Thumbnail'])
        if img_base64:
            body += f"<img src='data:image/jpeg;base64,{img_base64}' alt='Haber Resmi' style='max-width:100%; width:1000px; height:auto;'><br>"

        body += f"<p>{news['Summary']}</p>"
        body += f"<a href='{news['Link']}'>Haberi Oku</a><br>"
        body += f"<p><strong>Tarih:</strong> {news['Date']}</p><hr>"

    msg.attach(MIMEText(body, 'html'))

    # .eml dosyası olarak kaydetme
    eml_file_name = f"aselsan_bulteni_{datetime.now().strftime('%Y-%m-%d')}.eml"
    with open(eml_file_name, 'w') as f:
        f.write(msg.as_string())

    print(f"E-posta oluşturuldu ve kaydedildi: {eml_file_name}")

# Ana fonksiyon
def get_aselsan_news_report(start_date, end_date):
    news_links = search_aselsan_news(start_date, end_date)

    if news_links:
        news_data = []
        for news in news_links:
            print(f"Processing: {news['title']}")
            content = scrape_news_content(news['link'])
            if content:
                summary = summarize_content(content)  # OpenAI API ile özet
                news_data.append({
                    'Title': news['title'],
                    'Link': news['link'],
                    'Date': news['date'],
                    'Source': news['source'],  # Kaynak bilgisi eklendi
                    'Summary': summary,
                    'Thumbnail': news['thumbnail']  # Haber resmi eklendi
                })
        save_news_to_csv(news_data)
        create_eml_file(news_data, start_date, end_date)  # EML dosyası oluşturma
    else:
        print("No news found for the given date range.")

# Tarih aralığını belirtin
start_date = "09/1/2024"
end_date = "09/1/2024"

# Ana fonksiyonu çalıştır
get_aselsan_news_report(start_date, end_date)
